In [19]:
import os
import json
import math
import pandas as pd
from datetime import date, time
import numpy as np
import psycopg2

## MIGRACION DATOS A POSTGRESQL YELLOW DATA

In [2]:
# Establecer la semilla aleatoria para reproducibilidad
np.random.seed(123)

In [24]:
data = pd.read_csv('../datasets/processed/yellow_analytics/yellow_analytics_part_2.csv', sep=',', encoding='utf8')

In [25]:
data.head()

,IdProveedor,TotalPasajeros,DistanciaViaje,IdTipoTarifa,IdZonaOrigen,IdZonaDestino,IdTipoPago,CostoTotal,FechaRecogida,HoraRecogida,DuracionViaje
0,2,1,7.08,1.0,186,40,1,44.01,2022-01-09,18,1480
1,2,1,1.52,1.0,163,164,1,12.30,2022-01-09,18,451
2,1,1,11.10,1.0,132,92,2,34.05,2022-01-09,18,1658
3,2,1,2.10,1.0,48,238,1,13.80,2022-01-09,18,513
4,2,1,5.87,1.0,90,33,1,27.96,2022-01-09,18,1242


In [5]:
data.shape

(364131, 11)

In [6]:
data.IdProveedor.unique()

array([1, 2], dtype=int64)

In [7]:
data.IdProveedor.value_counts()

IdProveedor
2    256555
1    107576
Name: count, dtype: int64

In [26]:
#Calcular el tamaño de muestra proporcional para cada proveedor
proporciones = data['IdProveedor'].value_counts(normalize=True) * 0.12  # Proporción deseada
tamaño_muestra_por_proveedor = (proporciones * len(data)).astype(int)

# Lista para almacenar las muestras aleatorias estratificadas
muestras_estratificadas = []

# Iterar sobre cada proveedor único
for proveedor in data['IdProveedor'].unique():
    # Seleccionar aleatoriamente el tamaño de muestra proporcional para el proveedor actual
    muestra_proveedor = data[data['IdProveedor'] == proveedor].sample(tamaño_muestra_por_proveedor[proveedor])
    # Agregar las muestras seleccionadas al DataFrame de muestras estratificadas
    muestras_estratificadas.append(muestra_proveedor)

# Concatenar las muestras estratificadas en un solo DataFrame
muestras_estratificadas = pd.concat(muestras_estratificadas)

In [27]:
muestras_estratificadas.shape

(45534, 11)

In [28]:
muestras_estratificadas.IdProveedor.value_counts()

IdProveedor
2    32048
1    13486
Name: count, dtype: int64

In [11]:
muestras_estratificadas.head()

,IdProveedor,TotalPasajeros,DistanciaViaje,IdTipoTarifa,IdZonaOrigen,IdZonaDestino,IdTipoPago,CostoTotal,FechaRecogida,HoraRecogida,DuracionViaje
157178,1,1,2.1,1.0,238,48,1,14.55,2022-01-05,8,694
166028,1,1,2.5,1.0,68,229,1,17.75,2022-01-05,12,928
262756,1,1,1.0,1.0,107,164,1,12.35,2022-01-07,16,335
317951,1,3,2.3,1.0,161,246,1,14.80,2022-01-08,18,933
289658,1,2,5.5,1.0,114,238,1,27.35,2022-01-08,2,1247


In [12]:
muestras_estratificadas.dtypes


IdProveedor         int64
TotalPasajeros      int64
DistanciaViaje    float64
IdTipoTarifa      float64
IdZonaOrigen        int64
IdZonaDestino       int64
IdTipoPago          int64
CostoTotal        float64
FechaRecogida      object
HoraRecogida        int64
DuracionViaje       int64
dtype: object

In [29]:
# Conectar a la base de datos PostgreSQL
conn = psycopg2.connect(
    dbname="nycdb",
    user="postgres",
    password="admin",
    host="localhost"
)

In [30]:
# Crear un cursor para ejecutar comandos SQL
cur = conn.cursor()

In [31]:
muestras_estratificadas.columns

Index(['IdProveedor', 'TotalPasajeros', 'DistanciaViaje', 'IdTipoTarifa',
       'IdZonaOrigen', 'IdZonaDestino', 'IdTipoPago', 'CostoTotal',
       'FechaRecogida', 'HoraRecogida', 'DuracionViaje'],
      dtype='object')

In [16]:
muestras_estratificadas.dtypes

IdProveedor         int64
TotalPasajeros      int64
DistanciaViaje    float64
IdTipoTarifa      float64
IdZonaOrigen        int64
IdZonaDestino       int64
IdTipoPago          int64
CostoTotal        float64
FechaRecogida      object
HoraRecogida        int64
DuracionViaje       int64
dtype: object

In [32]:
# # Crear la tabla en la base de datos
# cur.execute("""
#     CREATE TABLE service_yellow (
#         Id SERIAL PRIMARY KEY,
#         IdProveedor INTEGER,
#         TotalPasajeros INTEGER,
#         DistanciaViaje FLOAT,
#         IdTipoTarifa FLOAT,
#         IdZonaOrigen INTEGER,
#         IdZonaDestino INTEGER,
#         IdTipoPago INTEGER,
#         CostoTotal FLOAT,
#         FechaRecogida DATE,
#         HoraRecogida INTEGER,
#         DuracionViaje INTEGER
#     )
# """)

# # Guardar los cambios en la base de datos
# conn.commit()


# Convertir los valores del DataFrame a listas de Python
registros = muestras_estratificadas.values.tolist()

# Insertar los registros en la tabla
cur.executemany("""
    INSERT INTO service_yellow (
        IdProveedor, TotalPasajeros, DistanciaViaje, IdTipoTarifa, IdZonaOrigen, IdZonaDestino, IdTipoPago, CostoTotal, FechaRecogida, HoraRecogida, DuracionViaje) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
""", registros)

# Guardar los cambios en la base de datos
conn.commit()

# Cerrar la conexión con la base de datos
cur.close()
conn.close()

print("Registros migrados exitosamente a PostgreSQL.")

Registros migrados exitosamente a PostgreSQL.


## MIGRACION DATOS A POSTGRESQL FHVHV DATA

In [20]:
# Establecer la semilla aleatoria para reproducibilidad
np.random.seed(123)

In [21]:
data = pd.read_csv('../datasets/processed/ffvh_analytics/ffvh_analytics_part_1.csv', sep=',', encoding='utf8')

In [22]:
data.head()

,IdProveedor,IdProveedor.1,IdZonaOrigen,IdZonaDestino,DistanciaViaje,trip_time,TarifaPasajero,CostoTotal,FechaRecogida,HoraRecogida,FechaLlegada,HoraLlegada,FechaSolicitada,HoraSolicitada,FechaAtendida,HoraAtendida,DuracionAtencion
0,HV0003,B03404,170,161,1.18,664,24.90,53.64,2022-01-01,00:07,2022-01-01,00:18,2022-01-01,00:05,2022-01-01,00:05,9.0
1,HV0003,B03404,237,161,0.82,460,11.97,28.46,2022-01-01,00:22,2022-01-01,00:30,2022-01-01,00:19,2022-01-01,00:22,161.0
2,HV0003,B03404,237,161,1.18,595,29.82,59.41,2022-01-01,00:57,2022-01-01,01:07,2022-01-01,00:43,2022-01-01,00:57,824.0
3,HV0003,B03404,262,229,1.65,303,7.91,17.90,2022-01-01,00:18,2022-01-01,00:23,2022-01-01,00:15,2022-01-01,00:17,92.0
4,HV0003,B03404,229,141,1.65,461,9.44,20.75,2022-01-01,00:28,2022-01-01,00:35,2022-01-01,00:25,2022-01-01,00:26,16.0


In [23]:
data.shape

(2665068, 17)

In [ ]:
#Calcular el tamaño de muestra proporcional para cada proveedor
proporciones = data['IdProveedor'].value_counts(normalize=True) * 0.12  # Proporción deseada
tamaño_muestra_por_proveedor = (proporciones * len(data)).astype(int)

# Lista para almacenar las muestras aleatorias estratificadas
muestras_estratificadas = []

# Iterar sobre cada proveedor único
for proveedor in data['IdProveedor'].unique():
    # Seleccionar aleatoriamente el tamaño de muestra proporcional para el proveedor actual
    muestra_proveedor = data[data['IdProveedor'] == proveedor].sample(tamaño_muestra_por_proveedor[proveedor])
    # Agregar las muestras seleccionadas al DataFrame de muestras estratificadas
    muestras_estratificadas.append(muestra_proveedor)

# Concatenar las muestras estratificadas en un solo DataFrame
muestras_estratificadas = pd.concat(muestras_estratificadas)

In [ ]:
muestras_estratificadas.shape

In [ ]:
muestras_estratificadas.IdProveedor.value_counts()

In [ ]:
muestras_estratificadas.head()

In [ ]:
muestras_estratificadas.dtypes

In [ ]:
# Conectar a la base de datos PostgreSQL
conn = psycopg2.connect(
    dbname="nycdb",
    user="postgres",
    password="admin",
    host="localhost"
)

In [ ]:
# Crear un cursor para ejecutar comandos SQL
cur = conn.cursor()

In [ ]:
# Crear la tabla en la base de datos
cur.execute("""
    CREATE TABLE service_yellow (
        Id SERIAL PRIMARY KEY,
        IdProveedor INTEGER,
        TotalPasajeros INTEGER,
        DistanciaViaje FLOAT,
        IdTipoTarifa FLOAT,
        IdZonaOrigen INTEGER,
        IdZonaDestino INTEGER,
        IdTipoPago INTEGER,
        CostoTotal FLOAT,
        FechaRecogida DATE,
        HoraRecogida INTEGER,
        DuracionViaje INTEGER
    )
""")

# Guardar los cambios en la base de datos
conn.commit()


# Convertir los valores del DataFrame a listas de Python
registros = muestras_estratificadas.values.tolist()

# Insertar los registros en la tabla
cur.executemany("""
    INSERT INTO service_yellow (
        IdProveedor, TotalPasajeros, DistanciaViaje, IdTipoTarifa, IdZonaOrigen, IdZonaDestino, IdTipoPago, CostoTotal, FechaRecogida, HoraRecogida, DuracionViaje) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
""", registros)

# Guardar los cambios en la base de datos
conn.commit()

# Cerrar la conexión con la base de datos
cur.close()
conn.close()

print("Registros migrados exitosamente a PostgreSQL.")